# Clasificación Naive Bayes
Este notebook implementa un modelo de Naive Bayes para clasificar estrellas como tipo **F** o **K** usando los atributos: Temperatura, Luminosidad y si es habitable.

In [1]:

import pandas as pd
import numpy as np
from math import exp, sqrt, pi

In [2]:
# Función gaussiana
def gaussiana(x, mu, sigma):
    return (1 / (sigma * sqrt(2 * pi))) * exp(-0.5 * ((x - mu) / sigma) ** 2)

In [3]:
# Diccionario con parámetros del modelo
modelo_nb = {
    "F": {
        "temperatura_mu": 5100.00,
        "temperatura_sigma": 2086.86,
        "habitable_si": 0.25,
        "habitable_no": 0.75,
        "luminosidad_mu": 11.75,
        "luminosidad_sigma": 7.63,
        "prior": 0.4
    },
    "K": {
        "temperatura_mu": 2866.67,
        "temperatura_sigma": 1231.08,
        "habitable_si": 0.5,
        "habitable_no": 0.5,
        "luminosidad_mu": 3.33,
        "luminosidad_sigma": 1.49,
        "prior": 0.6
    }
}


In [4]:
# Clasificador Naive Bayes para una estrella
def clasificar_estrella_prob(temperatura, habitable, luminosidad):
    resultados = []
    for clase in ["F", "K"]:
        p_temp = gaussiana(temperatura, modelo_nb[clase]["temperatura_mu"], modelo_nb[clase]["temperatura_sigma"])
        p_hab = modelo_nb[clase][f"habitable_{habitable.lower()}"]
        p_lum = gaussiana(luminosidad, modelo_nb[clase]["luminosidad_mu"], modelo_nb[clase]["luminosidad_sigma"])
        p_total = p_temp * p_hab * p_lum
        p_con_prior = p_total * modelo_nb[clase]["prior"]
        resultados.append({
            "Clase": clase,
            "P(x|Clase)": p_total,
            "P(x|Clase)*P(Clase)": p_con_prior
        })
    prediccion = max(resultados, key=lambda r: r["P(x|Clase)*P(Clase)"])["Clase"]
    return resultados[0], resultados[1], prediccion

# Datos de ejemplo
estrellas = [
    {"Temperatura": 1900, "Habitable": "No", "Luminosidad": 1},
    {"Temperatura": 2300, "Habitable": "No", "Luminosidad": 2},
    {"Temperatura": 5500, "Habitable": "No", "Luminosidad": 5}
]

In [5]:
# Clasificación
tabla_prob = []
for estrella in estrellas:
    f, k, pred = clasificar_estrella_prob(estrella["Temperatura"], estrella["Habitable"], estrella["Luminosidad"])
    tabla_prob.append({
        "T": estrella["Temperatura"],
        "H": estrella["Habitable"],
        "L": estrella["Luminosidad"],
        "P(x|F)": f["P(x|Clase)"],
        "P(x|K)": k["P(x|Clase)"],
        "P(x|F)*P(F)": f["P(x|Clase)*P(Clase)"],
        "P(x|K)*P(K)": k["P(x|Clase)*P(Clase)"],
        "Predicción": pred
    })

df_resultado = pd.DataFrame(tabla_prob)
pd.set_option("display.float_format", lambda x: f"{x:.2e}")
df_resultado


,T,H,L,P(x|F),P(x|K),P(x|F)*P(F),P(x|K)*P(K),Predicción
0,1900,No,1,8.58e-07,9.38e-06,3.43e-07,5.63e-06,K
1,2300,No,2,1.35e-06,2.62e-05,5.39e-07,1.57e-05,K
2,5500,No,5,4.98e-06,2.35e-06,1.99e-06,1.41e-06,F
